## Notebook to adjust PlanetScope images using the estimated surface reflectance of snow

Rainey Aberle

Summer 2022

__To-Do:__
- option to remove intermediate files (i.e., if hillshade correction and radiometric adjustment are selected, remove hs-corrected file)
- option to filter images with clipped bands

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
import rasterio.features
from shapely.geometry import Polygon
import shapely.geometry
from scipy.interpolate import interp2d
from scipy import stats
import time
import rasterio as rio
import geopandas as gpd
import sys
import ee

### Authenticate and initialize Google Earth Engine (GEE)

__Note:__ The first time you run the following cell, you will be asked to authenticate your GEE account for use in this notebook. This will send you to an external web page, where you will walk through the GEE authentication workflow and copy an authentication code back in this notebook when prompted. 

In [ ]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

### 1. Define paths in directory, snow-covered area (SCA), and settings

In [ ]:
# -----Define paths in directory
site_name = 'Sperry'
# path to planet-snow/ directory
base_path = '/Users/raineyaberle/Research/PhD/Planet_snow_cover/planet-snow/'
# path to stitched PlanetScope images
im_path = base_path+'../study-sites/'+site_name+'/imagery/PlanetScope/stitched/'
# output folder
out_path = im_path+'../adjusted-filtered/'
# AOI shapefile full path in directory
AOI_fn = base_path+'../../GIS_data/RGI_outlines/'+site_name+'_RGI.shp'

# -----Determine settings
plot_results = True # = True to plot corrected images
hillshade_correction = False # = True to apply hillshade correction using a DEM
skip_clipped = False # = True to skip images where bands appear "clipped", i.e. max blue SR < 0.8
adjust_radiometry = True # = True to adjust image band radiometry using the SCA

# -----Add path to functions
sys.path.insert(1, base_path+'functions/')
from image_adjustment_utils import sunpos, into_range, apply_hillshade_correction, create_top_elev_AOI_poly, adjust_image_radiometry
from classification_utils import query_GEE_for_DEM

### 2. Load AOI, DEM, and PlanetScope images, determine approximate year-round snow-covered area (SCA) based on elevations distribution

In [ ]:
# -----Load AOI as gpd.GeoDataFrame
AOI = gpd.read_file(AOI_fn)

# -----Load DEM
os.chdir(im_path) 
im_fns = glob.glob('*.tif') # read image filenames
im_fns.sort() # sort chronologically
DEM, DEM_x, DEM_y, AOI_UTM = query_GEE_for_DEM(AOI, im_path, im_fns)

# -----Create a polygon of the top 10th percentile elevations within the AOI
polygon, im_fn, im, r, g, b, im_x, im_y = create_top_elev_AOI_poly(AOI_UTM, im_path, im_fns, DEM, DEM_x, DEM_y)

# -----Plot
fig = plt.figure(figsize=(8,8))
plt.imshow(np.dstack([r, g, b]), extent=(np.min(im_x), np.max(im_x), np.min(im_y), np.max(im_y)))
plt.plot(*polygon.exterior.xy, color='orange', linewidth=2, label='polygon')
plt.xlabel('Easting [m]')
plt.ylabel('Northing [m]')
plt.title(im_fn)
fig.legend(loc='upper right')
fig.tight_layout()
plt.show()

### 3. Loop through images

In [ ]:
# -----Start timer
t1 = time.monotonic()

# -----Loop through images
for im_fn in im_fns:
    
    # load image
    print('Image: ', im_fn)
    im = rio.open(im_fn)
    
    # apply hillshade correction
    if (hillshade_correction == True):
        im_corrected_name = apply_hillshade_correction(CRS, polygon, im, im_fn, im_path, DEM_path, hs_path, out_path, skip_clipped, plot_results)
        if im_corrected_fn=='N/A':
            print('----------')
            print(' ')            
            continue
        else:
            im_corrected = rio.open(out_path + im_corrected_fn)
    
    # adjust radiometry
    if (adjust_radiometry == True):
        if (hillshade_correction == True):
            im_adj_fn = adjust_image_radiometry(im_corrected, im_corrected_fn, out_path, polygon, out_path, skip_clipped, plot_results)
        else:
            im_adj_fn = adjust_image_radiometry(im, im_fn, im_path, polygon, out_path, skip_clipped, plot_results)
    
    print('----------')
    print(' ')

# -----Stop timer
t2 = time.monotonic()
print('Time elapsed: ',str(round(t2-t1)/60),'minutes ')
    